In [ ]:
import gurobipy as gp
import numpy as np
import pandas as pd

In [ ]:
datos = pd.read_csv('PATH/TO/instancedata.csv')

find LCQPs

In [ ]:
datos = datos[datos['conscurvature'] == 'linear']
datos = datos.drop(columns=['conscurvature'])

remove the indefinite ones, the rest are all convex, no concave, no linear

In [ ]:
datos = datos[datos['objcurvature'] == 'convex']
datos = datos.drop(columns=['objcurvature', 'nobjquadnegev', 'convex'])

In [ ]:
datos = datos[~np.isnan(datos['solobjvalue'])]
# datos = datos[datos['solobjvalue'] != 0]
# datos = datos.drop(columns=['solobjvalue'])

In [ ]:
datos = datos.drop(columns=['nsos1', 'nsos2', 'nintvars', 'nquadfunc', 'objsense', 'objquadproblevfrac', 'njacobiannlnz', 'objtype', 'nnlfunc', 'nldensity', 'nnlsemi',
                           'ndiagquadcons', 'nnlintvars', 'nindefinitenlcons', 'solinfeasibility', 'nobjnz', 'nobjnlnz', 'nlnz', 'nz', 'njacobiannz',
                           'nlaghessiandiagnz', 'solsource', 'donor', 'nsemi', 'nquadcons', 'nobjquadnz', 'nlaghessiannz', 'nconvexnlcons', 'nlincons', 'nlinfunc',
                           'nobjquaddiagnz', 'laghessianmaxblocksize', 'nconcavenlcons', 'nlaghessianblocks', 'laghessianminblocksize', 'nobjquadposev',
                           'nnlbinvars', 'ncontvars', 'nnlvars', 'nbinvars', 'nsingleboundedvars', 'nboundedvars', 'laghessianavgblocksize'])

In [ ]:
datos

In [ ]:
target = 'QPLIB_8790'

In [ ]:
import gurobipy as gp
cat = np.concatenate

In [ ]:
def get_array(name):
    model = gp.read(f"PATH/TO/lp/{name}.lp")
    model = model.relax()
    assert np.all(np.array(model.getAttr("vtype", model.getVars())) == 'C')
    assert model.ModelSense == 1  # 1 for min, -1 for max

    A     = model.getA().todense()
    sense = np.array(model.getAttr("Sense", model.getConstrs()))
    b = np.array(model.getAttr("rhs", model.getConstrs()))
    
    Aeq = np.array(A[sense == '=', :])
    Ale = np.array(A[sense == '<', :])
    Age = np.array(A[sense == '>', :])
    
    beq = np.array(b[sense == '='])
    ble = np.array(b[sense == '<'])
    bge = np.array(b[sense == '>'])
    
    Ale = cat([Ale, -Age], axis=0)
    ble = cat([ble, -bge], axis=0)
    # sense[sense == '>'] = '<'

    model.setAttr("LB", model.getVars(), 0)
    model.setAttr("UB", model.getVars(), np.inf)
    model.update()

    lb = np.array(model.getAttr("LB", model.getVars()))
    ub = np.array(model.getAttr("UB", model.getVars()))

    Q = model.getQ().todense()
    c = np.array(model.getAttr("obj", model.getVars()))
    return Q, c, Aeq, Ale, beq, ble, lb, ub, model

In [ ]:
Q, c, Aeq, Ale, beq, ble, lb, ub, old_model = get_array(target)

In [ ]:
old_model.optimize()

In [ ]:
all_vars = old_model.getVars()
values = old_model.getAttr("X", all_vars)
solution = np.array(values)
obj = old_model.getObjective().getValue()

In [ ]:
Ale = cat([Ale, np.eye(Ale.shape[0])], axis=1)
Aeq = cat([Aeq, np.zeros((Aeq.shape[0], Ale.shape[0]))], axis=1)
A = cat([Ale, Aeq], axis=0)
b = cat([ble, beq], axis=0)

In [ ]:
P = np.zeros((A.shape[1], A.shape[1]))
P[:Q.shape[0], :Q.shape[1]] = Q
P /= np.abs(P).max()

In [ ]:
q = cat([c, np.zeros(Ale.shape[0])], axis=0)
q /= np.abs(q).max()

In [ ]:
model = gp.Model("qp")
x = model.addMVar(A.shape[1])
model.setObjective(0.5 * x @ P @ x + (x * q).sum())
model.addConstr(A @ x == b)
# m.optimize()

In [ ]:
model.presolve()

In [ ]:
model.optimize()

In [ ]:
all_vars = model.getVars()
values = model.getAttr("X", all_vars)
solution = np.array(values)
obj = model.getObjective().getValue()

In [ ]:
solution

In [ ]:
obj

In [ ]:
from generate_instances import normalize_cons

In [ ]:
A, b = normalize_cons(A, b)

In [ ]:
from scipy.linalg import qr

In [ ]:
lmat, _ = qr(A.T)

In [ ]:
m, n = A.shape
nulls = lmat[:, m:]

In [ ]:
np.abs(A @ nulls).max()

In [ ]:
nulls.T @ nulls

In [ ]:
np.abs(nulls).max(0).min()

In [ ]:
from scipy.optimize import linprog

In [ ]:
m, n = A.shape

In [ ]:
sol = linprog(c=np.concatenate([np.zeros(n), np.array([-1.])], axis=0), 
                      A_ub=np.concatenate([-np.eye(n), np.ones((n, 1))], axis=1), 
                      b_ub=np.zeros(n), 
                      A_eq=np.concatenate([A, np.zeros((m, 1))], axis=1), b_eq=b, 
                      # we set upper bound in case unbounded e.g. svm
                      bounds=(0., None), method='highs')

In [ ]:
sol.x.min()

In [ ]:
from torch_geometric.data import HeteroData
import torch

In [ ]:
A = torch.from_numpy(A).to(torch.float)
b = torch.from_numpy(b).to(torch.float)
q = torch.from_numpy(q).to(torch.float)
solution = torch.from_numpy(solution).to(torch.float)
x_feasible = torch.from_numpy(sol.x[:-1]).to(torch.float)

P = torch.from_numpy(P).to(torch.float)
A_where = torch.where(A)
P_where = torch.where(P)

data = HeteroData(
    cons={
        'num_nodes': A.shape[0],
        'x': torch.empty(A.shape[0]),
         },
    vals={
        'num_nodes': A.shape[1],
        'x': torch.empty(A.shape[1]),
    },
    # we create a tripartite graph, but we may NOT use the global node
    obj={
            'num_nodes': 1,
            'x': torch.zeros(1, 1).float(),
        },
    cons__to__vals={'edge_index': torch.vstack(A_where),
                    'edge_attr': A[A_where][:, None]},
    vals__to__vals={'edge_index': torch.vstack(P_where),
                    'edge_attr': P[P_where][:, None]},
    obj__to__vals={'edge_index': torch.vstack([torch.zeros(A.shape[1]).long(),
                                               torch.arange(A.shape[1])]),
                    'edge_attr': torch.ones(A.shape[1], 1).float()},
    obj__to__cons={'edge_index': torch.vstack([torch.zeros(A.shape[0]).long(),
                                               torch.arange(A.shape[0])]),
                    'edge_attr': torch.ones(A.shape[0], 1).float()},
    x_solution=solution,
    x_feasible=x_feasible,
    obj_solution=obj,
    b=b,
    q=q,
    nulls=torch.from_numpy(nulls).float().reshape(-1)
)

In [ ]:
torch.save(data, f'qplib_pt/{target}.pt')